# **Encapsulation**

Encapsulation is one of the four core principles of object-oriented programming (OOP). Encapsulation means **hiding internal data** from the outside world and only **allowing controlled access** via methods or interfaces.

**Important Note: In Python, encapsulation is by convention, not by enforcement.**


## **Why is it useful?**
1. Prevents unintended modification of data.
2. Allows validation before updating data.
3. Maintains integrity and security of the object's state.

## **How to Encapsulate in Python?**
In Python, we **prefix variables with an underscore** `_` to signal they are **"private"** (by convention). Python **doesn't enforce** strict access, but **developers are expected to respect it**.
```python
class Student:
    def __init__(self, age):
        self._age = age  # protected attribute
```
**But this alone doesn’t prevent wrong inputs!**

## **Use of single underscore vs double underscore**
The idea of the double underscore in Python is completely different. It was created as a means to override different methods of a class that is going to be extended several times, without the risk of having collisions with the method names. Even that is a too far-fetched use case as to justify the use of this mechanism.

Double underscores are a non-Pythonic approach. If we need to define attributes as private, use a single underscore, and respect the Pythonic convention that it is a private attribute.

Note that:
- Using __double_underscore triggers name mangling — not true privacy.
- But using _single_underscore is considered "Pythonic" — it indicates something is private, not enforces it.

## **Private vs Name Mangling vs Magic Methods**

| Symbol     | Purpose                                                        |
| ---------- | -------------------------------------------------------------- |
| `_name`    | You want to mark an attribute as private                       |
| `__name`   | Name Mangling: Avoid name clashes in inheritance.              |
| `__name__` | Special/Magic methods (e.g., `__init__`, `__str__`) i.e. dunder|

## **Name Mangling**
When you name an attribute like **__my_var** inside a class, Python internally changes its name to **_ClassName__my_var**.

This is called name mangling, and it's used to prevent accidental overrides when classes are extended (i.e., inheritance).

In [1]:
class Student:
    def __init__(self, age):
        self._age = age  # private attribute

s = Student(-20)

print(s._age) # Python doesn't enforce strict access

-20


## **Getters and Setters - @property and @attr.setter Decorator**
- In OOP, objects are created to model real-world things.
- These objects often contain data (like attributes) and behaviors (like methods).
- Data accuracy/validity is critical — e.g., age should not be negative, email should be properly formatted.
- So we write validation logic, especially in setters.
- In Python, instead of writing separate get_x() and set_x() methods like in Java, we use @property — a cleaner, Pythonic way to handle this.
- Keeps the interface clean (like obj.age) instead of calling obj.get_age().


#### **What is @property?**  
The @property decorator in Python is used to define a method as a getter for a class attribute, allowing you to access it like a regular attribute while still including logic or validation behind the scenes. It is part of Python’s way of supporting encapsulation in an elegant and Pythonic way.

@property makes a method look like an attribute. @property is mandatory before using @attr.setter.

| Decorator      | Purpose                                   | Mandatory?                     |
| -------------- | ----------------------------------------- | ------------------------------ |
| `@property`    | Turns a method into a **getter** property | Yes                          |
| `@attr.setter` | Adds a **setter** to the property `attr`  | Yes — only after `@property` |

#### **Why is @property mandatory before @attr.setter?**  
- In Python, the @attr.setter decorator is not standalone.
- It extends an existing property object.
- That property object is first created using the @property decorator.

#### **What is you skip @property?**  
If you try to use @attr.setter without first defining @property, you'll get: 
```
AttributeError: 'function' object has no attribute 'setter'
``` 
Because age was never defined as a @property in the first place, so there's nothing to extend with .setter

#### **How does it work internally?**  
When you use:
```python
@property
def age(self):
    return self._age
```
You are creating a property object named **age**. That object knows how to get the value.

Now you can **extend** that property object to include setter behavior like follows:
```python
@age.setter
def age(self, value):
    self._age = value
```

In [5]:
class Person:
    def __init__(self, age):
        self.age = age   # This triggers @age.setter

    @property
    def age(self):
        print("Getter called")
        return self._age
    
    @age.setter
    def age(self, value):
        print("Setter called")
        if value < 0:
            raise ValueError("Age cannot be negative")
        self._age = value

In [6]:
p = Person(25)     # Call the setter
print(p.age)       # Call the getter
print(p.__dict__)

Setter called
Getter called
25
{'_age': 25}


In [7]:
p.age = 30         # Call the setter
print(p.age)       # Call the getter
print(p.__dict__)

Setter called
Getter called
30
{'_age': 30}


In [8]:
p.age = -1         # Call the setter
print(p.age)       # Call the getter
print(p.__dict__)

Setter called


ValueError: Age cannot be negative

## **Real Time Example of Encapsulation**

Consider the example of a geographical system that needs to deal with coordinates. There is only a certain range of values for which latitude and longitude make sense. Outside of those values, a coordinate cannot exist. We can create an object to represent a coordinate, but in doing so we must ensure that the values for latitude are at all times within the acceptable ranges. And for this, we can use properties:

In [9]:
class Coordinate:
    def __init__(self, lat: float, long: float) -> None:
        self._latitude = self._longitude = None
        self.latitude = lat       # This triggers @latitude.setter
        self.longitude = long     # This triggers @longitude.setter

    @property
    def latitude(self) -> float:
        return self._latitude

    @latitude.setter
    def latitude(self, lat_value: float) -> None:
        if lat_value not in range(-90, 90 + 1):
            raise ValueError(f"{lat_value} is an invalid value for latitude")
        self._latitude = lat_value

    @property 
    def longitude(self) -> float: 
        return self._longitude

    @longitude.setter
    def longitude(self, long_value: float) -> None:
        if long_value not in range(-180, 180 + 1):
            raise ValueError(f"{long_value} is an invalid value for longitude")
        self._longitude = long_value